# Transform Custom Data

## 1. Remove records with null customer_id

In [0]:
%sql
SELECT *
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL;

## 2. Remove exact duplicates records

In [0]:
%sql
SELECT *
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL
ORDER BY customer_id;

In [0]:
%sql
SELECT DISTINCT *
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL
ORDER BY customer_id;

## 3. Remove duplicate records based on created_timestamp

Si qusieramos tomar valores únicos según la última fecha de ceración

In [0]:
%sql
WITH cte_ultimos_updates AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY created_timestamp DESC) AS rn
    FROM gizmobox.bronze.v_customers
    WHERE customer_id IS NOT NULL
)
SELECT *
FROM cte_ultimos_updates
WHERE rn = 1;

## 4. CAST the column values to the correct data type

In [0]:
%sql
WITH cte_ultimos_updates AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY created_timestamp DESC) AS rn
    FROM gizmobox.bronze.v_customers
    WHERE customer_id IS NOT NULL
)
SELECT
    CAST(created_timestamp AS timestamp) AS created_timestamp,
    customer_id,
    customer_name,
    CAST(date_of_birth AS DATE) AS date_of_birth,
    email,
    CAST(member_since AS DATE) AS member_since,
    telephone,
    file_path
FROM cte_ultimos_updates
WHERE rn = 1;

## 5. Write Data to Delta Table

In [0]:
%sql
CREATE TABLE gizmobox.silver.customers
AS
WITH cte_ultimos_updates AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY created_timestamp DESC) AS rn
    FROM gizmobox.bronze.v_customers
    WHERE customer_id IS NOT NULL
)
SELECT
    CAST(created_timestamp AS timestamp) AS created_timestamp,
    customer_id,
    customer_name,
    CAST(date_of_birth AS DATE) AS date_of_birth,
    email,
    CAST(member_since AS DATE) AS member_since,
    telephone,
    file_path
FROM cte_ultimos_updates
WHERE rn = 1;

In [0]:
%sql
SELECT * FROM gizmobox.silver.customers;